In [17]:
!pip install "git+https://github.com/jemisjoky/TorchMPS.git"
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchmps import MPS

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jemisjoky/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-sfbgpf7l
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-sfbgpf7l'


In [18]:
# Get the data

# Data hyperparameters
nb_train = 200
nb_test = 50
chosen_bs = 100

# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, sampler = test_subset, batch_size = chosen_bs)

In [19]:
# Create and train the FCNN

# Pick the hyperparameters for the FCNN
chosen_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_epochs_fcnn = 1
hidden_size = 70
chosen_loss = nn.CrossEntropyLoss()
#Optimizer parameters
chosen_lr = 0.001
chosen_momentum = 0.9


# Instantiate the fcnn
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.lin1 = nn.Linear(784, hidden_size)
        self.relu1 = nn.ReLU()

        self.lin2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()

        self.lin3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()

        self.lin4 = nn.Linear(hidden_size, hidden_size)
        self.relu4 = nn.ReLU()

        self.lin5 = nn.Linear(hidden_size, hidden_size)
        self.relu5 = nn.ReLU()

        self.lin6 = nn.Linear(hidden_size, 10)

    def forward(self, x):
        y = self.lin1(x)
        y = self.relu1(y)

        y = self.lin2(y)
        y = self.relu2(y)

        y = self.lin3(y)
        y = self.relu3(y)

        y = self.lin4(y)
        y = self.relu4(y)

        y = self.lin5(y)
        y = self.relu5(y)

        y = self.lin6(y)
        return y

#Instantiate and put the model on the chosen device
fcnn_teacher = FCNN().to(chosen_device)

#Instantiate the optimizer
optimizer = torch.optim.Adam(fcnn_teacher.parameters())

#Training loop
for epoch in range(n_epochs_fcnn):
    for (x_mb, y_mb) in train_iterator:

        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        # Foward propagation
        y_hat_mb = fcnn_teacher(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(loss.item())


2.3106789588928223


In [20]:
# Print the accuracy of the fcnn
with torch.no_grad():
    acc_count = 0
    for (x_mb, y_mb) in test_iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        y_mb_hat = fcnn_teacher(x_mb) #predict

        (wedontcare, y_mb_hat_class) = torch.max(y_mb_hat, 1)
        acc_count += (y_mb_hat_class == y_mb).sum().item()

    print("Accuracy: " + str(100* acc_count / len(test_iterator.dataset)))

Accuracy: 0.04


In [22]:
# Create and train the MPS
# MPS parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False

# Training parameters
gn_var = 0.3 #gaussian noise variance
gn_mean = 0 #gaussiane noise mean
n_epochs_lmps = 15
learn_rate = 0.0001
l2_reg = 0.0
chosen_loss = nn.CrossEntropyLoss()

# Initialize the MPS module
mps_student = MPS(
    input_dim=28 ** 2,
    output_dim=10,
    bond_dim=bond_dim,
    adaptive_mode=adaptive_mode,
    periodic_bc=periodic_bc,
).to(chosen_device)

#Instantiate the optimizer
lmps_optimizer = torch.optim.Adam(mps_student.parameters())

#Training loop
for epoch in range(n_epochs_lmps):
    for (x_mb, _) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        x_mb = x_mb + torch.randn(size=x_mb.size()).to(chosen_device) #add gaussian noise to inputs
        x_mb = x_mb.to(chosen_device)
        y_mb = fcnn_teacher(x_mb).to(chosen_device)

        # Foward propagation
        y_hat_mb = mps_student(x_mb)
        loss = chosen_loss(y_hat_mb, y_mb)

        # Backpropagation
        loss.backward()
        lmps_optimizer.step()
        lmps_optimizer.zero_grad()

    print(loss.item())

# Print the accuracy of the mps_student
with torch.no_grad():
    acc_count = 0
    for (x_mb, y_mb) in test_iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        y_mb_hat = mps_student(x_mb) #predict

        (wedontcare, y_mb_hat_class) = torch.max(y_mb_hat, 1)
        acc_count += (y_mb_hat_class == y_mb).sum().item()

    print("Accuracy: " + str(100* acc_count / len(test_iterator.dataset)))


0.1187450960278511
-11056513.0
-1081133060063232.0
-9.06428508217769e+22
-1.515134673565195e+23
-1.8227739428150077e+23
-1.9374319864481587e+23
-2.3888223727799506e+23
-1.6773667620780518e+23
-1.7521448905788817e+23
-1.8006720772837243e+23
-2.126213875628275e+23
-1.9448872452713078e+23
-1.934338013504155e+23
-2.3065891655999567e+23
Accuracy: 0.05
